# Implementation of AutoKeras

In [0]:
import os
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from PIL import Image
from keras.datasets import mnist
from keras.models import Sequential
from keras import regularizers
from keras.datasets import cifar10
#from keras.regularizers import WeightRegularizer, ActivityRegularizer 
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization 
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
#from plotter import Plotter
# from keras.utils.visualize_util import plot
import h5py
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
def get_annotations_map():
	valAnnotationsPath = 'tiny-imagenet-200/tiny-imagenet-200/val/val_annotations.txt'
	valAnnotationsFile = open(valAnnotationsPath, 'r')
	valAnnotationsContents = valAnnotationsFile.read()
	valAnnotations = {}

	for line in valAnnotationsContents.splitlines():
		pieces = line.strip().split()
		valAnnotations[pieces[0]] = pieces[1]

	return valAnnotations

In [0]:
def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,32,32],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')

    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,32,32],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass



    return X_train,y_train,X_test,y_test

In [0]:
from google.colab import files
uploaded=files.upload()


Saving tiny-imagenet-200.zip to tiny-imagenet-200.zip


In [0]:
import zipfile
import io
data = zipfile.ZipFile(io.BytesIO(uploaded['tiny-imagenet-200.zip']), 'r')
data.extractall()


In [0]:
train_data_dir = 'tiny-imagenet-200/tiny-imagenet-200/train'  
validation_data_dir = 'tiny-imagenet-200/tiny-imagenet-200/val'  
test_data_dir = 'tiny-imagenet-200//tiny-imagenet-200test' 
target_names = [item for item in os.listdir(train_data_dir) if os.path.isdir(os.path.join(train_data_dir, item))]
nb_train_samples = sum([len(files) for _, _, files in os.walk(train_data_dir)])  
nb_validation_samples = sum([len(files) for _, _, files in os.walk(validation_data_dir)])
nb_test_samples = sum([len(files) for _, _, files in os.walk(test_data_dir)])
total_nb_samples = nb_train_samples + nb_validation_samples + nb_test_samples

nb_classes = len(target_names)      # number of output classes

print('Training a CNN Multi-Classifier Model ......')
print('\n - names of classes: ', target_names, '\n - # of classes: ', nb_classes)
print(' - # of trained samples: ', nb_train_samples, '\n - # of validation samples: ', nb_validation_samples,
      '\n - # of test samples: ', nb_test_samples,
       '\n - total # of samples: ', total_nb_samples, '\n - train ratio:', round(nb_train_samples/total_nb_samples*100, 2),
      '\n - validation ratio:', round(nb_validation_samples/total_nb_samples*100, 2),
      '\n - test ratio:', round(nb_test_samples/total_nb_samples*100, 2),
     ' %', '\n - # of epochs: ', 2, '\n - batch size: ', 500)

Training a CNN Multi-Classifier Model ......

 - names of classes:  ['n02129165', 'n03837869', 'n01742172', 'n03250847', 'n03126707', 'n02815834', 'n04532670', 'n03854065', 'n12267677', 'n07615774', 'n03649909', 'n03584254', 'n02226429', 'n02423022', 'n03014705', 'n09332890', 'n03617480', 'n02206856', 'n03763968', 'n03201208', 'n01774384', 'n01698640', 'n01944390', 'n04507155', 'n04532106', 'n03400231', 'n03796401', 'n04259630', 'n02883205', 'n02403003', 'n03976657', 'n02837789', 'n04251144', 'n02123045', 'n02795169', 'n01855672', 'n02124075', 'n04540053', 'n04067472', 'n04560804', 'n02056570', 'n01983481', 'n02486410', 'n07920052', 'n04074963', 'n04376876', 'n04118538', 'n04465501', 'n03838899', 'n03404251', 'n04597913', 'n02988304', 'n03160309', 'n02504458', 'n04417672', 'n02410509', 'n02909870', 'n06596364', 'n07734744', 'n03983396', 'n02480495', 'n02281406', 'n04371430', 'n03662601', 'n03544143', 'n07875152', 'n03937543', 'n02927161', 'n01644900', 'n07720875', 'n02841315', 'n026994

In [0]:
path='tiny-imagenet-200/tiny-imagenet-200'
X_train,y_train,X_test,y_test=load_images(path,nb_classes)

Loading 200 classes
loading training images...
finished loading training images
loading test images...


In [0]:
!pip install autokeras

    100% |████████████████████████████████| 102kB 7.7MB/s 
    100% |████████████████████████████████| 26.6MB 1.4MB/s 
    100% |████████████████████████████████| 560.1MB 27kB/s 
    100% |████████████████████████████████| 13.9MB 2.4MB/s 
    100% |████████████████████████████████| 25.3MB 1.4MB/s 
    100% |████████████████████████████████| 51kB 18.4MB/s 
    100% |████████████████████████████████| 83.1MB 371kB/s 
    100% |████████████████████████████████| 61kB 23.7MB/s 
    100% |████████████████████████████████| 8.9MB 3.1MB/s 
    100% |████████████████████████████████| 1.6MB 8.2MB/s 
    100% |████████████████████████████████| 1.4MB 5.2MB/s 
    100% |████████████████████████████████| 143kB 31.8MB/s 
    100% |████████████████████████████████| 25.4MB 1.6MB/s 
    100% |████████████████████████████████| 2.0MB 6.8MB/s 
    100% |████████████████████████████████| 706kB 9.0MB/s 
  Stored in directory: /root/.cache/pip/wheels/20/b7/59/3a1011287e69e559553751f400a6b3874448e94f66cb0745ba
 

In [0]:
!pip install imageio

In [0]:
!pip install GPUtil

In [0]:
from autokeras.image.image_supervised import ImageClassifier


In [0]:
from autokeras import CnnModule
from autokeras.nn.loss_function import classification_loss
from autokeras.nn.metric import Accuracy

TEST_FOLDER = "test"
cnnModule = CnnModule(loss=classification_loss, metric=Accuracy, searcher_args={}, path=TEST_FOLDER, verbose=False)


In [0]:
cnnModule.fit(n_output_node, input_shape, train_data, test_data, time_limit=24 * 60 * 60)


In [0]:
clf = ImageClassifier(path="~/automodels/",verbose=True, augment=False) # Give a custom path of your choice
clf.fit(X_train, y_train, time_limit=4*60*60)
clf.final_fit(X_train, y_train, X_test, y_test, retrain=True)

Saving Directory: ~/automodels/
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   15.962398386001587   |  0.14440000000000003   |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|               Training model 1               |
+----------------------------------------------+
                                      